# Proyecto Final - Ensemble Learning (Aprendizaje por ensamblado)
## Parte 2 - Implementación y predicción

**Curso:** Statistical Learning

**Catedrático:** Ing. Luis Leal

**Estudiante:** Dany Rafael Díaz Lux (21000864)

**Objetivo:** Hacer clasificación binaria para determinar si una persona sobrevive o no del hundimiento del Titanic.

In [137]:
# Import required libraries
#from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
#import datetime as dt
import joblib
#import matplotlib.pylab as plt
import numpy as np 
#import os.path
import pandas as pd
import scipy.stats as st
import sklearn.metrics as mts
#import tensorflow as tf
#print('Tensor flow version: ' + tf.__version__)

In [238]:
# Informacion de modelos
directorioModelos = 'modelos/'
extensionModelos = '.modelo'
nombreModeloArbolDecision = 'ArbolDecision_criterio=gini_profundidadMaxima=4'
nombreModeloSVM = 'SVM_regularizacionC=1.0_kernel=poly_grado=3'
nombreParametrosRegresionLogistica = 'RegresionLogisticaMiniBatchGradientDescent_epochs=500_lr=0.01_batchSize=32_alpha=0.0'
nombreProbabilidadesNaiveBayes = 'NaiveBayes'

In [239]:
# Cargar modelos
modeloArbolDecision = joblib.load(directorioModelos + nombreModeloArbolDecision + extensionModelos)
modeloSVM = joblib.load(directorioModelos + nombreModeloSVM + extensionModelos)
parametrosRegresionLogistica = np.load(directorioModelos + nombreParametrosRegresionLogistica + extensionModelos + '.npy')
probabilidadesNaiveBayes = dict(np.load(directorioModelos + nombreProbabilidadesNaiveBayes + extensionModelos + '.npy', allow_pickle=True)[0])

In [240]:
# Funciones auxiliares para modelo de regresión logística
def agregarColumnaUnos(x):
    if len(x.shape) < 2:
        x = np.array(x, dtype=np.float32).reshape(x.shape[0], -1)
    return np.append(x, np.ones(x.shape[0]).reshape(-1,1), 1)

def calcularSigmoid(x):
    return 1/(1 + np.exp(-x))

def predecirRegresionLogistica(X, parametros):
    X = agregarColumnaUnos(X)
    logits = np.matmul(X, parametros)
    logits = logits.reshape(logits.shape[0])
    return np.round(calcularSigmoid(logits))

# Función que indicará si una feature en X es categórica o no
def esCategorica(X, indexFeature):
    if(type(X).__module__ != np.__name__):
        X = np.array(X)
        
    if(len(np.unique(X[:,indexFeature])) < 11 and\
        len(np.unique(X[:,indexFeature]))/len(X[:,5]) < 0.4):
        return True
    return False

def predecirNaiveBayes(diccionarioProbabilidades, X):
    if(type(X).__module__ != np.__name__):
        X = np.array(X)
    etiquetas = diccionarioProbabilidades['__labels__']
    y_pred = np.zeros(shape=(X.shape[0],1), dtype=np.float32)
    for i in range(X.shape[0]):
        resultados = np.ones_like(etiquetas, dtype=np.float32)
        for indiceEtiqueta, etiqueta in enumerate(etiquetas):
            resultados[indiceEtiqueta] = diccionarioProbabilidades[str(etiqueta)]
            for indiceCaracteristica in range(X.shape[1]):
                if(esCategorica(X, indiceCaracteristica)):
                    if(str(indiceCaracteristica) + '_' + str(round(X[i,indiceCaracteristica], 10)) + '|' + str(etiqueta) \
                        in diccionarioProbabilidades.keys()):
                        resultados[indiceEtiqueta] *= \
                            diccionarioProbabilidades[str(indiceCaracteristica) + '_' + \
                                                      str(round(X[i,indiceCaracteristica], 10)) + '|' + str(etiqueta)]
                    else:
                        resultados[indiceEtiqueta] = 0.0
                        break
                # Para características continuas se obtiene la probabilidad del valor (en un rango pequeño) para valor dado
                else:
                    if('media_' + str(indiceCaracteristica) + '|' + str(etiqueta) \
                        in diccionarioProbabilidades.keys()):
                        margen = 0.001
                        valorZ = (X[i,indiceCaracteristica] \
                                  - diccionarioProbabilidades['media_' + str(indiceCaracteristica) + '|' + str(etiqueta)])\
                                    / diccionarioProbabilidades['desviacion_' + str(indiceCaracteristica) + '|' + str(etiqueta)]
                        probabilidad = st.norm.cdf(valorZ + margen) - st.norm.cdf(valorZ - margen)
                        resultados[indiceEtiqueta] *= probabilidad

        y_pred[i,0] = etiquetas[np.argmax(resultados)]
    return y_pred

In [241]:
pruebas = pd.read_csv("datos_validacion_temp.csv")
caracteristicas = pruebas.iloc[:,:-1]
etiquetas = pruebas.iloc[:,-1]

In [242]:
# Revisar métricas de datos de prueba
prediccionesArbol = modeloArbolDecision.predict(caracteristicas)
print(mts.accuracy_score(etiquetas, prediccionesArbol))
print(mts.f1_score(etiquetas, prediccionesArbol))
print(mts.precision_score(etiquetas, prediccionesArbol))
print(mts.recall_score(etiquetas, prediccionesArbol))

0.8321678321678322
0.8095238095238094
0.8225806451612904
0.796875


In [243]:
# Revisar métricas de datos de prueba
prediccionesSVM = modeloSVM.predict(caracteristicas)
print(mts.accuracy_score(etiquetas, prediccionesSVM))
print(mts.f1_score(etiquetas, prediccionesSVM))
print(mts.precision_score(etiquetas, prediccionesSVM))
print(mts.recall_score(etiquetas, prediccionesSVM))

0.8181818181818182
0.7936507936507936
0.8064516129032258
0.78125


In [244]:
# Revisar métricas de datos de prueba
prediccionesRegresion = predecirRegresionLogistica(caracteristicas, parametrosRegresionLogistica)
print(mts.accuracy_score(etiquetas, prediccionesRegresion))
print(mts.f1_score(etiquetas, prediccionesRegresion))
print(mts.precision_score(etiquetas, prediccionesRegresion))
print(mts.recall_score(etiquetas, prediccionesRegresion))

0.8391608391608392
0.8130081300813008
0.847457627118644
0.78125


In [246]:
# Revisar métricas de datos de prueba
prediccionesNaiveBayes = predecirNaiveBayes(probabilidadesNaiveBayes, caracteristicas)
print(mts.accuracy_score(etiquetas, prediccionesNaiveBayes))
print(mts.f1_score(etiquetas, prediccionesNaiveBayes))
print(mts.precision_score(etiquetas, prediccionesNaiveBayes))
print(mts.recall_score(etiquetas, prediccionesNaiveBayes))

0.7762237762237763
0.7460317460317459
0.7580645161290323
0.734375


## Función de predicción

In [253]:
# Predicción final combinando los modelos: Árbol de decisión, SVM
def prediccionFinal(X):
    prediccionesArbolDecision = modeloArbolDecision.predict(X)
    prediccionesSVM = modeloSVM.predict(X)
    prediccionesRegresionLogistica = predecirRegresionLogistica(X, parametrosRegresionLogistica)
    prediccionesNaiveBayes = predecirNaiveBayes(probabilidadesNaiveBayes, X).reshape(X.shape[0])
    prediccionTestArbolDecision = 0.8322
    prediccionTestSVM = 0.8182
    prediccionTestRegresionLogistica = 0.8130
    prediccionTestNaiveBayes = 0.7762
    totalPrediccion = prediccionTestArbolDecision + prediccionTestSVM + prediccionTestRegresionLogistica + prediccionTestNaiveBayes
    pesoArbolDecision =  prediccionTestArbolDecision / totalPrediccion
    pesoSVM = prediccionTestSVM / totalPrediccion
    pesoRegresionLogistica = prediccionTestRegresionLogistica / totalPrediccion
    pesoNaiveBayes = prediccionTestNaiveBayes / totalPrediccion
    print(prediccionesArbolDecision.shape, prediccionesSVM.shape, prediccionesRegresionLogistica.shape, prediccionesNaiveBayes.shape)
    prediccionFinal = pesoArbolDecision * prediccionesArbolDecision \
                    + pesoSVM * prediccionesSVM \
                    + pesoRegresionLogistica * prediccionesRegresionLogistica \
                    + pesoNaiveBayes * prediccionesNaiveBayes
    return np.round(prediccionFinal)

In [254]:
# Revisar métricas de datos de prueba
prediccionesFinales = prediccionFinal(caracteristicas)
print(prediccionesFinales)
print(mts.accuracy_score(etiquetas, prediccionesFinales))
print(mts.f1_score(etiquetas, prediccionesFinales))
print(mts.precision_score(etiquetas, prediccionesFinales))
print(mts.recall_score(etiquetas, prediccionesFinales))

(143,) (143,) (143,) (143,)
[0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1.
 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
0.8251748251748252
0.8
0.819672131147541
0.78125


In [107]:
modeloNaiveBayes = np.load('test.npy', allow_pickle=True)
#diccTest = dict(enumerate(modeloNaiveBayes[0].flatten(), 1))[1]
#diccTest['0_0']
modeloNaiveBayes[0]['1']

0.4

## Predicción árbol de decisión

## Predicción Support Vector Machine (SVM)

## Predicción Naive Bayes

## Predicción regresión logística

## Validación